In [1]:
# Imports

import tensorflow as tf
import numpy as np
import os
import pathlib
import IPython
import sys
!pip install keras_tuner -q
import keras_tuner
!pip install git+https://github.com/themis-ai/capsa.git
import capsa
from capsa import EnsembleWrapper
!pip install helper
import helper

from google.colab import drive             
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/themis-ai/capsa.git to /tmp/pip-req-build-diy7agxd
  Running command git clone --filter=blob:none --quiet https://github.com/themis-ai/capsa.git /tmp/pip-req-build-diy7agxd
  Resolved https://github.com/themis-ai/capsa.git to commit d5f0decf7e8d4abdb9ca370a0b9bb03787de8e9a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')
data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')

dataset_path = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)


Found 4159 files belonging to 2 classes.
Using 3328 files for training.
Found 4159 files belonging to 2 classes.
Using 831 files for validation.
Found 921 files belonging to 2 classes.


In [ ]:
# # catch and prefetch
# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
#classification cModel

num_classes = 1


model = tf.keras.Sequential([
    tf.keras.Input(shape=(64, 64, 3)),
    # tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, strides = 2, padding = "same",  activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    # tf.keras.layers.Dropout(0.5),                                                                         #DP1
    tf.keras.layers.Conv2D(32, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.5),                                                                         #DP2
    tf.keras.layers.Conv2D(64, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, 1, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.7),                                                                         #DP3
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 512, activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP4                                                                        #DP5
    tf.keras.layers.Dense(num_classes, activation= None)
])


# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
#     metrics=[tf.keras.metrics.BinaryAccuracy()], 
#     run_eagerly=True
# )

# model.fit(
#         dataset_path,
#         epochs=6,
#         batch_size=64,
#   )

model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 16)        448       
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 4, 128)         8320      
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_10 (Dense)            (None, 512)               1049088   
                                                                 
 dense_11 (Dense)            (None, 1)                

In [15]:
# the HistogramVAEWrapper check

decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(units=4*4*128),  
        tf.keras.layers.Reshape((4, 4, 128)),
        tf.keras.layers.Conv2DTranspose(64, 1,  strides=2, padding = "same"),
        tf.keras.layers.Conv2DTranspose(32, 3,  strides=2, padding = "same"),
        tf.keras.layers.Conv2DTranspose(16, 3,  strides=2, padding = "same"),
        tf.keras.layers.Conv2DTranspose(3, 3,  strides=2, padding = "same"),
    ])



wrapped_model = capsa.HistogramVAEWrapper(model, num_bins=5, queue_size=20000, latent_dim = 64, decoder=decoder)

wrapped_model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.00001),
    loss= tf.losses.BinaryCrossentropy(from_logits= True),
    metrics= [tf.keras.metrics.BinaryAccuracy()]
)
# wrapped_model.summary()

history = wrapped_model.fit(
    dataset_path,
    epochs=10,                                    
    validation_data = dataset_path_val)


out = wrapped_model.predict(dataset_path_test, batch_size=512)


Epoch 1/10


52/52 [==============================] - 15s 160ms/step - vae_compiled_loss: nan - vae_compiled_binary_accuracy: 0.5319 - vae_wrapper_loss: nan - val_vae_compiled_loss: nan - val_vae_compiled_binary_accuracy: 0.5018 - val_vae_wrapper_loss: nan
Epoch 2/10
52/52 [==============================] - 7s 123ms/step - vae_compiled_loss: nan - vae_compiled_binary_accuracy: 0.5247 - vae_wrapper_loss: nan - val_vae_compiled_loss: nan - val_vae_compiled_binary_accuracy: 0.5018 - val_vae_wrapper_loss: nan
Epoch 3/10
52/52 [==============================] - 9s 147ms/step - vae_compiled_loss: nan - vae_compiled_binary_accuracy: 0.5393 - vae_wrapper_loss: nan - val_vae_compiled_loss: nan - val_vae_compiled_binary_accuracy: 0.5018 - val_vae_wrapper_loss: nan
Epoch 4/10
52/52 [==============================] - 8s 146ms/step - vae_compiled_loss: nan - vae_compiled_binary_accuracy: 0.5344 - vae_wrapper_loss: nan - val_vae_compiled_loss: nan - val_vae_compiled_binary_accuracy: 0.5018 - val_vae_wrapper_loss

In [9]:
mve_model = capsa.MVEWrapper(model, is_classification=True)

mve_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=[tf.keras.metrics.BinaryAccuracy()], 
    run_eagerly=True
)

history = mve_model.fit(
        dataset_path,
        epochs=6,
        batch_size=64,
  )

normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = dataset_path.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds = dataset_path_test.map(lambda x, y: (normalization_layer(x), y))

images = np.vstack((normalized_test_ds, normalized_train_ds))

# print(type(images[1][0]))

# images= tf.convert_to_tensor(np.asarray(images).astype('float32'))

# predictions = model(images)

Epoch 1/6


52/52 [==============================] - 9s 92ms/step - mve_compiled_loss: nan - mve_compiled_binary_accuracy: 0.5345 - mve_wrapper_loss: nan
Epoch 2/6
52/52 [==============================] - 7s 116ms/step - mve_compiled_loss: nan - mve_compiled_binary_accuracy: 0.5295 - mve_wrapper_loss: nan
Epoch 3/6
52/52 [==============================] - 6s 97ms/step - mve_compiled_loss: nan - mve_compiled_binary_accuracy: 0.5330 - mve_wrapper_loss: nan
Epoch 4/6
52/52 [==============================] - 7s 121ms/step - mve_compiled_loss: nan - mve_compiled_binary_accuracy: 0.5367 - mve_wrapper_loss: nan
Epoch 5/6
52/52 [==============================] - 6s 93ms/step - mve_compiled_loss: nan - mve_compiled_binary_accuracy: 0.5215 - mve_wrapper_loss: nan
Epoch 6/6
52/52 [==============================] - 6s 94ms/step - mve_compiled_loss: nan - mve_compiled_binary_accuracy: 0.5212 - mve_wrapper_loss: nan
<class 'tensorflow.python.data.ops.map_op._MapDataset'>


In [ ]:
Ensemble_wrapped_model = EnsembleWrapper(model,num_members=1)


Ensemble_wrapped_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
                     )

history = Ensemble_wrapped_model.fit(
        dataset_path,
        epochs=12,
        batch_size=64,
  )

# mesh_grid = helper.get_grid()
# output = Ensemble_wrapped_model(mesh_grid)
# helper.plot_ensemble_classification(output,mesh_grid,dataset_path_test)

Epoch 1/12
52/52 [==============================] - 10s 104ms/step - usermodel_0_compiled_loss: 7.7131 - usermodel_0_compiled_binary_accuracy: 0.4879
Epoch 2/12
52/52 [==============================] - 6s 95ms/step - usermodel_0_compiled_loss: 7.9382 - usermodel_0_compiled_binary_accuracy: 0.4784
Epoch 3/12
52/52 [==============================] - 7s 117ms/step - usermodel_0_compiled_loss: 7.9419 - usermodel_0_compiled_binary_accuracy: 0.4786
Epoch 4/12
52/52 [==============================] - 6s 97ms/step - usermodel_0_compiled_loss: 7.9440 - usermodel_0_compiled_binary_accuracy: 0.4786
Epoch 5/12
52/52 [==============================] - 7s 124ms/step - usermodel_0_compiled_loss: 7.9457 - usermodel_0_compiled_binary_accuracy: 0.4786
Epoch 6/12
52/52 [==============================] - 6s 97ms/step - usermodel_0_compiled_loss: 7.9440 - usermodel_0_compiled_binary_accuracy: 0.4788
Epoch 7/12
52/52 [==============================] - 6s 96ms/step - usermodel_0_compiled_loss: 7.9463 - userm

AttributeError: ignored

In [ ]:
from capsa import DropoutWrapper

base_model = unet(drop_prob=0.1)
# don't add dropout in the wrapper because our
# model already contains dropout layers
model = DropoutWrapper(base_model, p=0.0)